<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [1]:
import os 

## Configuration
class MyConfig:
    pass 

MY_CONFIG = MyConfig ()

## Input Data - configure this to the folder we want to process
MY_CONFIG.INPUT_DATA_DIR = "input"
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  1
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

### Download Data

We will use [Walmart annual report PDFs](https://github.com/sujee/data/tree/main/data-prep-kit/walmart-reports-1) as our input data.

Feel free to substitute your data

In [2]:
import os, sys
import shutil
from utils import download_file

## Download the data files
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart-10K-Reports-Optimized_2023.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart-10K-Reports-Optimized_2023.pdf' ))

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart_2024.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart_2024.pdf' ))

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart_2024.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart_2024_copy.pdf' ))  # create a dupe file

Local file 'input/Walmart-10K-Reports-Optimized_2023.pdf' (1.61 MB) already exists. Skipping download.
Local file 'input/Walmart_2024.pdf' (4.87 MB) already exists. Skipping download.
Local file 'input/Walmart_2024_copy.pdf' (4.87 MB) already exists. Skipping download.


### Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")


## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


### Import Common python modules

In [4]:
import os
import sys

# Main repo root
from utils import rootdir

from data_processing_ray.runtime.ray import RayTransformLauncher
from data_processing.runtime.pure_python import PythonTransformLauncher
from data_processing.utils import ParamsUtils

STAGE = 0

<a id="pdf2parquet"></a>

## Step-2: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### Set Input/output Folder

In [5]:
STAGE  += 1
# STAGE = 1  ## DEBUG

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_parquet_out")

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'


### Execute 

In [6]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_code_location": ParamsUtils.convert_to_ast(code_location),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


08:01:15 INFO - Running locally
08:01:15 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'double_precision': 8}
08:01:15 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
08:01:15 INFO - data factory data_ max_files -1, n_sample -1
08:01:15 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
08:01:15 INFO - pipeline id pipeline_id
08:01:15 INFO - code location {'github': 'github', 'commit_hash': '12345', 'path': 'path'}
08:01:15 INFO - number of workers 2 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
08:01:15 INFO - actor creation delay 0
08:01:15 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job_id'}
202

Progress: |--------------------------------------------------| 0.1% Complete
Progress: |--------------------------------------------------| 0.2% Complete
Progress: |--------------------------------------------------| 0.2% Complete
Progress: |--------------------------------------------------| 0.5% Complete
Progress: |--------------------------------------------------| 0.6% Complete
Progress: |--------------------------------------------------| 0.9% Complete
Progress: |--------------------------------------------------| 1.0% Complete
Progress: |--------------------------------------------------| 0.8% Complete
Progress: |--------------------------------------------------| 1.3% Complete
Progress: |--------------------------------------------------| 1.1% Complete
Progress: |--------------------------------------------------| 1.8% Complete
Progress: |--------------------------------------------------| 1.4% Complete
Progress: |█-------------------------------------------------| 2.2% Complete

(RayTransformFileProcessor pid=86180) Downloading recognition model, please wait. This may take several minutes depending upon your network connection.
(RayTransformFileProcessor pid=86181) 08:01:21 INFO - Initializing models
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 51509.00it/s]
(RayTransformFileProcessor pid=86181) Downloading detection model, please wait. This may take several minutes depending upon your network connection.


Progress: |--------------------------------------------------| 1.7% Complete
Progress: |███████████████████████████████████████████-------| 87.4% Complete [repeated 7x across cluster]
Progress: |███████████████████████████████████████████-------| 87.2% Complete
Progress: |██------------------------------------------------| 4.4% Complete
Progress: |--------------------------------------------------| 1.8% Complete
Progress: |██------------------------------------------------| 4.6% Complete
Progress: |--------------------------------------------------| 1.9% Complete
Progress: |██------------------------------------------------| 4.8% Complete
Progress: |█-------------------------------------------------| 2.0% Complete
Progress: |██------------------------------------------------| 5.0% Complete
Progress: |██------------------------------------------------| 5.1% Complete
Progress: |██------------------------------------------------| 5.2% Complete
Progress: |██--------------------------------

(orchestrate pid=86175) 08:01:53 ERROR - Got Ray worker exception The actor died because of an error raised in its creation task, ray::RayTransformFileProcessor.__init__() (pid=86180, ip=127.0.0.1, actor_id=76a65cb8cc5f6862c4f6601d01000000, repr=<data_processing_ray.runtime.ray.transform_file_processor.RayTransformFileProcessor object at 0x109f22f50>)
(orchestrate pid=86175)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(orchestrate pid=86175)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(orchestrate pid=86175)   File "/Users/touma/data-prep-kit/examples/notebooks/rag/venv/lib/python3.11/site-packages/data_processing_ray/runtime/ray/transform_file_processor.py", line 46, in __init__
(orchestrate pid=86175)     self.transform = params.get("transform_class", None)(self.transform_params)
(orchestrate pid=86175)                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(orchestrate pid=86175)   File "/Users/touma/data-prep-kit/examples/notebooks/rag/venv/lib/python3.1

Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████--------| 85.5% Complete [repeated 7x across cluster]


08:02:03 INFO - Completed execution in 0.808 min, execution result 1
(RayTransformFileProcessor pid=86181) Downloading recognition model, please wait. This may take several minutes depending upon your network connection.
(RayTransformFileProcessor pid=86181)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ [repeated 2x across cluster]
(RayTransformFileProcessor pid=86181)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ [repeated 2x across cluster]
(RayTransformFileProcessor pid=86181)   File "/Users/touma/data-prep-kit/examples/notebooks/rag/venv/lib/python3.11/site-packages/easyocr/easyocr.py", line 178, in __init__ [repeated 14x across cluster]
(RayTransformFileProcessor pid=86181)     self.transform = params.get("transform_class", None)(self.transform_params) [repeated 2x across cluster]
(RayTransformFileProcessor pid=86181)                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ [repeated 2x across cluster]
(RayTransformFileProcessor pid=86181)     super().__init_

Progress: |███████████████████████████████████████████-------| 86.8% Complete
Progress: |████████████████████████████████████████████------| 89.2% Complete
Progress: |████████████████████████████████████████████------| 89.0% Complete [repeated 2x across cluster]
Progress: |██████████████------------------------------------| 28.5% Complete [repeated 4x across cluster]
Progress: |███████████████████-------------------------------| 38.8% Complete [repeated 2x across cluster]
Progress: |█████████████████████-----------------------------| 44.0% Complete [repeated 2x across cluster]
Progress: |█████████████████████████████████-----------------| 66.5% Complete
Progress: |█████████████████████████████████████████████-----| 91.6% Complete [repeated 2x across cluster]
Progress: |██████████████████████████████████████████████----| 93.0% Complete [repeated 4x across cluster]
Progress: |███████████████████████████████████████-----------| 79.0% Complete [repeated 6x across cluster]


Exception: ❌ Ray job failed

### Inspect Generated output

Here we should see one entry per input file processed

In [7]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

ValueError: No objects to concatenate

<a id="chunking"></a>

##  Step-3: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### Set Input/output Folder

In [ ]:
STAGE  += 1
# STAGE = 2  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_chunk_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

### Execute 

In [ ]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

### Inspect Generated output

We would see documents are split into many chunks

In [ ]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(3)

## Step-4: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### Set Input/output Folder

In [ ]:
STAGE  += 1
# STAGE  = 3  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_ededupe_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

### Execute 

In [ ]:
%%time

# Import ededup transform configuration
## MT ###########################################################
#from ededup_transform_ray import EdedupRayTransformConfiguration
from ededup_transform_ray import EdedupRayTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
## MT ###########################################################
#launcher = RayTransformLauncher(EdedupRayTransformConfiguration())
launcher = RayTransformLauncher(EdedupRayTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

### Inspect Generated output

In [ ]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

## MT ###########################################################
#output_df.sample(3)
output_df.sample(len(output_df))

## Step-5:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

In [ ]:

# Input for this stage is the output of exact dedeup component
# output of this component makes it possible for fdedup component to run on data.

STAGE  += 1
# STAGE  = 4  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_doc_id_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

In [ ]:
%%time 

## MT ###########################################################
from doc_id_transform_ray import DocIDRayTransformConfiguration
from doc_id_transform_ray import DocIDRayTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "hash_column",
    "doc_id_int_column": "int_id_column",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

## MT ###########################################################
#launcher = RayTransformLauncher(DocIDRayTransformConfiguration())
launcher = RayTransformLauncher(DocIDRayTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

### Inspect Generated output

In [ ]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

## MT ###########################################################
#output_df.sample(3)
output_df.sample(len(output_df))

## Step-6: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### Set Input/output Folder

In [ ]:
## Input to this component is the output of doc_id generator component. 

STAGE  += 1
# STAGE  = 5  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_fdedupe_out")
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

### Execute 

In [ ]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "int_id_column",
    "fdedup_cluster_column": "hash_column",
    # infrastructure
    "fdedup_bucket_cpu": 0.5,
    "fdedup_doc_cpu": 0.5,
    "fdedup_mhash_cpu": 0.5,
    "fdedup_num_doc_actors": 2,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 2,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.8,
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

### Inspect Generated output

In [ ]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

## MT ###########################################################
#output_df.sample(3)
output_df.sample(len(output_df))

## Step-7: Document Quality

### Set Input/output Folder

In [ ]:
STAGE  += 1
# STAGE  = 6 ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_doc_quality_out")
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

### Execute 

In [ ]:
%%time

import os
import sys
from pathlib import Path

from doc_quality_transform import (
    text_lang_cli_param,
    doc_content_column_cli_param,
    bad_word_filepath_cli_param,
)
from doc_quality_transform_ray import DocQualityRayTransformConfiguration
from data_processing.utils import ParamsUtils

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

doc_quality_basedir = os.path.join(rootdir, "transforms", "language", "doc_quality", "python")
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_creation_delay": 0,
    # doc quality configuration
    text_lang_cli_param: "en",
    doc_content_column_cli_param: "contents",
    bad_word_filepath_cli_param: os.path.join(doc_quality_basedir, "ldnoobw", "en"),
}


Path(output_folder).mkdir(parents=True, exist_ok=True)

sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocQualityRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

### Inspect Generated output

In [ ]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

## MT ###########################################################
#output_df.sample(3)
output_df.sample(len(output_df))

## Step-8:   Text encoding

Encode text for the vector storage.

In [ ]:
STAGE  += 1
# STAGE  = 7 ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_encoder_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

In [ ]:
%%time 

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

### Inspect Generated output

In [ ]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

## MT ###########################################################
#output_df.sample(3)
output_df.sample(len(output_df))

## Step-9: Copy output to final output dir

In [ ]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")